# D-SAIL'2025: Our Coding Adventure
## Using LLMs to Adapt the Narrative of Tangible Programming Robot for Preschoolers

This script downloads the models and queries them. This script is available at https://github.com/mapto/our-coding-adventure and presented at the D-SAIL Workshop - Transformative Curriculum Design: Digitalisation, Sustainability, and AI Literacy for 21st Century Learning (https://d-sail.itd.cnr.it)

A separate script converts the generated markdown output into a minipage-based overview.

Install and load dependencies

In [ ]:
!pip install llama-cpp-python==0.3.9 huggingface-hub==0.31.4

In [ ]:
from llama_cpp import Llama

Specify which models to use. As it can be seen here, all used models are quantisations published by https://huggingface.co/bartowski

In [ ]:
models = [
    {
        "repo": "bartowski/gemma-2-9b-it-GGUF",
        "file": "gemma-2-9b-it-Q3_K_XL.gguf",
    },
    {
        "repo": "bartowski/OLMo-2-1124-7B-Instruct-GGUF",
        "file": "OLMo-2-1124-7B-Instruct-Q3_K_XL.gguf",
    },
    {
        "repo": "bartowski/Mistral-7B-Instruct-v0.3-GGUF",
        "file": "Mistral-7B-Instruct-v0.3-Q3_K_L.gguf",
    },
    {
        "repo": "bartowski/Qwen2.5-7B-Instruct-GGUF",
        "file": "Qwen2.5-7B-Instruct-Q3_K_XL.gguf",
    },
    {
        "repo": "bartowski/Meta-Llama-3.1-8B-Instruct-GGUF",
        "file": "Meta-Llama-3.1-8B-Instruct-Q3_K_XL.gguf",
    },
]

Specify prompt and parameters

In [ ]:
prompt_templ = """Suggest to a teacher a game with the Cubetto tangible programming toy for preschool children. It should be about {world}, involve {objects} and children should use Cubetto to {task}. Only three Cubetto command blocks should be used: forward, turn left and turn right. The description should be about half a page long."""

params = [
    {
        "world": "pirates",
        "objects": "Lego figures",
        "task": "find a treasure",
    },
    {
        "world": "Wild West",
        "objects": "Brio trains",
        "task": "rescue someone",
    },
    {
        "world": "superheroes",
        "objects": "Hot Wheels cars",
        "task": "enact a struggle",
    },
    {
        "world": "knights and princesses",
        "objects": "Barbie dolls",
        "task": "enact a pursuit",
    },
]

For each model and parameter set, fill the parameters in the prompt, feed it to the model, and store composed prompt and response.

In [ ]:
for m in models:
    mname = m["repo"].split("/")[1].split("-")[0]
    print(mname)
    llm = Llama.from_pretrained(
        repo_id=m["repo"],
        filename=m["file"],
        n_gpu_layers=-1,
        flash_attn=True,
    )
    for p in params:
        prompt = prompt_templ.format(**p)
        msg = [{"role": "user", "content": prompt}]
        output = llm.create_chat_completion(messages=msg, temperature=0)
        result = output["choices"][0]["message"]["content"]
        fname = f'results/{mname}-{p["world"]}-{p["objects"]}-{p["task"]}.txt'
        with open(fname, "w") as fout:
            fout.write(f"{m['repo'].split('/')[1]}\n{prompt}\n\n-------------------\n{result}")